In [ ]:
!pip install mysql-connector-python

In [2]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Umaneethi@123",
    database="zomato_deliveries"
)

cursor = conn.cursor()


In [ ]:
# Create Database
cursor.execute("CREATE DATABASE IF NOT EXISTS Zomato_Deliveries;")

In [3]:
# Connect to the newly created database
conn.database = "Zomato_Deliveries"

In [ ]:
from faker import Faker   
import random
from datetime import datetime, timedelta

# Initialize Faker
fake = Faker()

# Create Customers Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Customers (
    customer_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    email VARCHAR(255),
    phone VARCHAR(20),
    location VARCHAR(255),
    signup_date DATE,
    is_premium BOOLEAN,
    preferred_cuisine VARCHAR(50),
    total_orders INT,
    average_rating FLOAT
);
''')

# Create Restaurants Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Restaurants (
    restaurant_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    cuisine_type VARCHAR(50),
    location VARCHAR(255),
    owner_name VARCHAR(255),
    average_delivery_time INT,
    contact_number VARCHAR(20),
    rating FLOAT,
    total_orders INT,
    is_active BOOLEAN
);
''')

# Create Orders Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Orders (
    order_id INT AUTO_INCREMENT PRIMARY KEY,
    customer_id INT,
    restaurant_id INT,
    order_date DATETIME,
    delivery_time DATETIME,
    status VARCHAR(50),
    total_amount FLOAT,
    payment_mode VARCHAR(50),
    discount_applied FLOAT,
    feedback_rating FLOAT,
    FOREIGN KEY (customer_id) REFERENCES Customers(customer_id),
    FOREIGN KEY (restaurant_id) REFERENCES Restaurants(restaurant_id)
);
''')

# Create DeliveryPersons Table
cursor.execute('''
CREATE TABLE IF NOT EXISTS DeliveryPersons (
    delivery_person_id INT AUTO_INCREMENT PRIMARY KEY,
    name VARCHAR(255),
    contact_number VARCHAR(20),
    vehicle_type VARCHAR(50),
    total_deliveries INT,
    average_rating FLOAT,
    location VARCHAR(255)
);
''')

# Create Deliveries Table (Referencing DeliveryPersons)
cursor.execute('''
CREATE TABLE IF NOT EXISTS Deliveries (
    delivery_id INT AUTO_INCREMENT PRIMARY KEY,
    order_id INT,
    delivery_person_id INT,
    delivery_status VARCHAR(50),
    distance FLOAT,
    delivery_time INT,
    estimated_time INT,
    delivery_fee FLOAT,
    vehicle_type VARCHAR(50),
    FOREIGN KEY (order_id) REFERENCES Orders(order_id),
    FOREIGN KEY (delivery_person_id) REFERENCES DeliveryPersons(delivery_person_id)
);
''')

# Commit table creation
conn.commit()

In [ ]:
# Number of records to generate
num_customers = 100
num_restaurants = 20
num_orders = 150
num_delivery_persons = 30
num_deliveries = 100

# Insert Customers
def generate_customers(n):
    for _ in range(n):
        cursor.execute("""
            INSERT INTO Customers (name, email, phone, location, signup_date, is_premium, preferred_cuisine, total_orders, average_rating)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            fake.name(),
            fake.email(),
            fake.phone_number()[:20],
            fake.address(),
            fake.date_this_decade(),
            random.choice([True, False]),
            random.choice(['Indian', 'Chinese', 'Italian', 'Mexican', 'Thai']),
            random.randint(1, 50),
            round(random.uniform(1, 5), 1)
        ))

generate_customers(num_customers)

# Insert Restaurants
def generate_restaurants(n):
    for _ in range(n):
        cursor.execute("""
            INSERT INTO Restaurants (name, cuisine_type, location, owner_name, average_delivery_time, contact_number, rating, total_orders, is_active)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            fake.company(),
            random.choice(['Indian', 'Chinese', 'Italian', 'Mexican', 'Thai']),
            fake.city(),
            fake.name(),
            random.randint(20, 60),
            fake.phone_number()[:20],
            round(random.uniform(1, 5), 1),
            random.randint(50, 500),
            random.choice([True, False])
        ))

generate_restaurants(num_restaurants)

# Insert Orders
def generate_orders(n):
    for _ in range(n):
        order_date = fake.date_time_this_year()
        delivery_time = order_date + timedelta(minutes=random.randint(20, 90))
        cursor.execute("""
            INSERT INTO Orders (customer_id, restaurant_id, order_date, delivery_time, status, total_amount, payment_mode, discount_applied, feedback_rating)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            random.randint(1, num_customers),
            random.randint(1, num_restaurants),
            order_date,
            delivery_time,
            random.choice(['Pending', 'Delivered', 'Cancelled']),
            round(random.uniform(100, 1000), 2),
            random.choice(['Credit Card', 'Cash', 'UPI']),
            round(random.uniform(0, 50), 2),
            round(random.uniform(1, 5), 1)
        ))

generate_orders(num_orders)

# Insert Delivery Persons
def generate_delivery_persons(n):
    for _ in range(n):
        cursor.execute("""
            INSERT INTO DeliveryPersons (name, contact_number, vehicle_type, total_deliveries, average_rating, location)
            VALUES (%s, %s, %s, %s, %s, %s)
        """, (
            fake.name(),
            fake.phone_number()[:20],
            random.choice(['Bike', 'Car', 'Scooter']),
            random.randint(50, 500),
            round(random.uniform(1, 5), 1),
            fake.city()
        ))

generate_delivery_persons(num_delivery_persons)

# Insert Deliveries
def generate_deliveries(n):
    for _ in range(n):
        cursor.execute("""
            INSERT INTO Deliveries (order_id, delivery_person_id, delivery_status, distance, delivery_time, estimated_time, delivery_fee, vehicle_type)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (
            random.randint(1, num_orders),
            random.randint(1, num_delivery_persons),
            random.choice(['On the way', 'Delivered']),
            round(random.uniform(1, 15), 1),
            random.randint(20, 90),
            random.randint(15, 80),
            round(random.uniform(10, 100), 2),
            random.choice(['Bike', 'Car', 'Scooter'])
        ))

generate_deliveries(num_deliveries)

# Commit changes and close connection
conn.commit()
conn.close()

print("Dummy data successfully inserted into 'Zomato_Deliveries' database!")

In [4]:
# 1)1. Find the top 5 customers who placed the most orders.

query = """
SELECT c.customer_id, c.name, COUNT(o.order_id) AS total_orders 
FROM Customers c
JOIN Orders o ON c.customer_id = o.customer_id 
GROUP BY c.customer_id, c.name 
ORDER BY total_orders DESC 
LIMIT 5;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)



[(44, 'Julian Sherman', 4), (38, 'Megan Gregory', 4), (95, 'Dr. James Davis', 4), (36, 'Ryan Lee', 4), (52, 'Ann Yates', 4)]


In [5]:
#2. Find the most popular cuisine type among premium customers.

query = """
SELECT preferred_cuisine, COUNT(*) AS count 
FROM Customers 
WHERE is_premium = 1 
GROUP BY preferred_cuisine 
ORDER BY count DESC 
LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[('Thai', 16)]


In [5]:
#3. Find the top 3 highest-rated restaurants.

query = """
SELECT name, rating 
FROM Restaurants 
ORDER BY rating DESC 
LIMIT 3;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[('Ramos, Smith and Escobar', 4.8), ('Soto-Kim', 4.7), ('Adams Inc', 4.7)]


In [6]:
#4)Find the restaurant with the longest average delivery time.

query = """
SELECT name, average_delivery_time 
FROM Restaurants 
ORDER BY average_delivery_time DESC 
LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)

[('Soto-Kim', 60)]


In [7]:
#5. Find the top 5 customers who have spent the most money.

query = """
SELECT c.customer_id, c.name, SUM(o.total_amount) AS total_spent
FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id
GROUP BY c.customer_id, c.name
ORDER BY total_spent DESC
LIMIT 5;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)



[(44, 'Julian Sherman', 2803.6700134277344), (95, 'Dr. James Davis', 2535.3199768066406), (43, 'Michael Smith', 2460.4700317382812), (30, 'Barbara Anderson', 2452.510009765625), (32, 'Parker Simmons', 2378.679931640625)]


In [8]:
#6. Find the number of orders per payment mode.

query = """
SELECT payment_mode, COUNT(*) AS total_orders 
FROM Orders 
GROUP BY payment_mode;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[('UPI', 53), ('Cash', 43), ('Credit Card', 54)]


In [9]:
#7. Identify customers who have not placed any orders.

query = """
SELECT c.customer_id, c.name 
FROM Customers c 
LEFT JOIN Orders o ON c.customer_id = o.customer_id 
WHERE o.order_id IS NULL;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[(3, 'Kelli Hernandez'), (11, 'Edward Lambert'), (15, 'Justin Glenn'), (21, 'Sara Thompson'), (23, 'Kimberly Brown'), (33, 'Tammy Medina'), (35, 'Ryan Jackson'), (46, 'Laura James'), (48, 'Steven Holmes'), (50, 'Veronica Haas'), (53, 'Erica Gibson'), (56, 'Joseph Knapp'), (59, 'Joseph Banks'), (66, 'Mark Perez'), (68, 'Christopher Terrell'), (70, 'Brittney Price'), (71, 'Samantha Christensen'), (75, 'Joseph Lawrence'), (88, 'Sonya Mcdonald'), (89, 'Lisa Rodriguez'), (93, 'Melissa Charles'), (96, 'Eric Baker'), (99, 'Heather Flores'), (100, 'Brianna Cohen')]


In [10]:
#8. Find the most common delivery status.

query = """
SELECT delivery_status, COUNT(*) AS count 
FROM Deliveries 
GROUP BY delivery_status 
ORDER BY count DESC 
LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[('On the way', 50)]


In [11]:
#9. Find the top 3 most frequent customers (by order count).

query = """
SELECT c.customer_id, c.name, COUNT(o.order_id) AS total_orders
FROM Customers c
JOIN Orders o ON c.customer_id = o.customer_id
GROUP BY c.customer_id, c.name
ORDER BY total_orders DESC
LIMIT 3;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[(44, 'Julian Sherman', 4), (36, 'Ryan Lee', 4), (38, 'Megan Gregory', 4)]


In [12]:
#10. Identify the busiest delivery person.

query = """
SELECT d.delivery_person_id, d.name, COUNT(*) AS total_deliveries 
FROM Deliveries del
JOIN DeliveryPersons d ON del.delivery_person_id = d.delivery_person_id
GROUP BY d.delivery_person_id, d.name 
ORDER BY total_deliveries DESC 
LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[(2, 'Bradley Howell', 6)]


In [13]:
# 11.Find the restaurant with the best average feedback rating.

query = """
SELECT r.name, AVG(o.feedback_rating) AS avg_rating
FROM Orders o
JOIN Restaurants r ON o.restaurant_id = r.restaurant_id
GROUP BY r.name
ORDER BY avg_rating DESC
LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[('Hawkins Ltd', 4.333333412806193)]


In [14]:
#12. Find the highest-rated delivery person.

query = """
SELECT name, average_rating 
FROM DeliveryPersons 
ORDER BY average_rating DESC 
LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[('Jonathan Brooks', 4.6)]


In [15]:
#13. Identify the average discount applied per order.

query = """
SELECT AVG(discount_applied) AS avg_discount 
FROM Orders;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[(24.006266639431317,)]


In [16]:
# 14. Find the total revenue generated by each payment mode.

query = """
SELECT payment_mode, SUM(total_amount) AS total_revenue
FROM Orders
GROUP BY payment_mode
ORDER BY total_revenue DESC;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[('UPI', 29597.589920043945), ('Credit Card', 28882.669998168945), ('Cash', 24861.16000366211)]


In [17]:
# 15.Find the delivery person who has delivered the most orders.

query = """
SELECT d.delivery_person_id, d.name, COUNT(del.delivery_id) AS total_deliveries
FROM Deliveries del
JOIN DeliveryPersons d ON del.delivery_person_id = d.delivery_person_id
GROUP BY d.delivery_person_id, d.name
ORDER BY total_deliveries DESC
LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[(2, 'Bradley Howell', 6)]


In [18]:
# 16. Identify customers who have placed orders at more than 3 different restaurants.

query = """
SELECT c.customer_id, c.name, COUNT(DISTINCT o.restaurant_id) AS unique_restaurants 
FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id
GROUP BY c.customer_id, c.name 
HAVING unique_restaurants > 3;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[(36, 'Ryan Lee', 4), (38, 'Megan Gregory', 4), (44, 'Julian Sherman', 4), (52, 'Ann Yates', 4), (95, 'Dr. James Davis', 4)]


In [19]:
# 17.  Find the busiest time period (hour) for order placements.

query = """
SELECT HOUR(order_date) AS order_hour, COUNT(*) AS order_count 
FROM Orders 
GROUP BY order_hour 
ORDER BY order_count DESC 
LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[(8, 10)]


In [20]:
# 18. Find customers who have given an average rating above 4.5.

query = """
SELECT c.name, AVG(o.feedback_rating) AS avg_rating 
FROM Orders o
JOIN Customers c ON o.customer_id = c.customer_id
GROUP BY c.name 
HAVING avg_rating > 4.5;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[('Helen Johnson', 4.800000190734863), ('Marie Farrell', 4.699999809265137), ('Laura Hamilton', 4.599999904632568), ('Paul King', 4.599999904632568), ('Michelle Baird', 4.800000190734863), ('Mark Mathews', 4.900000095367432)]


In [21]:
# 19. Calculate total revenue per restaurant.

query = """
SELECT r.name, SUM(o.total_amount) AS total_revenue 
FROM Orders o
JOIN Restaurants r ON o.restaurant_id = r.restaurant_id
GROUP BY r.name 
ORDER BY total_revenue DESC;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)



[('Thomas-Green', 6745.729919433594), ('Thompson Group', 5773.7900390625), ('Singleton PLC', 5699.269958496094), ('Diaz-Frederick', 5676.130035400391), ('Campbell-Taylor', 5259.899971008301), ('Morris-Herrera', 5002.570007324219), ('Ashley, Morrison and Schmitt', 4596.420013427734), ('Owens Group', 4555.779968261719), ('Smith-Williams', 4308.919952392578), ('Shah-Gomez', 3978.149948120117), ('Wilson Group', 3898.4300231933594), ('Juarez-Harris', 3728.2599487304688), ('Cherry, Schultz and Webb', 3721.300048828125), ('Soto-Kim', 3643.280029296875), ('Ramos, Smith and Escobar', 3552.1000366210938), ('Larson Ltd', 3443.76993560791), ('Hall, Chapman and Hill', 2979.710006713867), ('Adams Inc', 2781.1600189208984), ('Hawkins Ltd', 2015.7000122070312), ('Kemp Group', 1981.050048828125)]


In [24]:
# 20.Find the restaurant with the highest repeat customer rate.

query = """
SELECT r.name, 
       COUNT(o.customer_id) AS total_orders, 
       COUNT(DISTINCT o.customer_id) AS unique_customers,
       (COUNT(o.customer_id) / COUNT(DISTINCT o.customer_id)) AS repeat_rate
FROM Orders o
JOIN Restaurants r ON o.restaurant_id = r.restaurant_id
GROUP BY r.name
ORDER BY repeat_rate DESC
LIMIT 1;
"""
cursor.execute(query)
result = cursor.fetchall()
print(result)


[('Soto-Kim', 7, 6, Decimal('1.1667'))]
